In [5]:
from PIL import Image
from pytesseract import pytesseract
import pyautogui
import os
import webbrowser
import pandas as pd
import cv2
import numpy as np
from pytesseract import Output
import imutils
import string 
from thefuzz import fuzz
from Levenshtein import distance, ratio


def create_tesseract_path():
    current_dir = os.getcwd()
    path_to_tesseract = r"\\Tesseract-OCR\tesseract.exe" #will fail in jupyter folder
    
    pytesseract.tesseract_cmd = path_to_tesseract


def clean_word(word):
    for char in word:
        if char in string.punctuation:
            word = word.replace(char, "")
            
    for char in word:
        if not char.isalnum():
            word = word.replace(char, "")
            
    return word.strip().lower()


def clean_df(df):
    df['text'] = df['text'].apply(lambda x: clean_word(x))
    
    for i, row in df.iterrows():
        if row['text'] == "":
            df.at[i, 'text'] = np.nan
            
    df.dropna(inplace=True)
    df.reset_index(inplace=True)
    
    #pad dataframe for later use
    for _ in range(3):
        df = df.append(pd.Series("endofdataframe", index=df.columns), ignore_index=True)
        
    return df

    
def preprocesser(file_path):
    """Allows for different preprocessing techniques to be added
    onto our input image to improve tesseract"""
    
    base_image = cv2.imread(file_path)
    
    #Current Process Inverts then changes to grayscale
    inverted_image = cv2.bitwise_not(base_image)
    gray_image = cv2.cvtColor(inverted_image, cv2.COLOR_BGR2GRAY)
    
    return gray_image


def process_image(file_path):
    psm_version = "--psm 11"
    
    preprocessed_image = preprocesser(file_path)
    image_df = pd.DataFrame(pytesseract.image_to_data(preprocessed_image, output_type=Output.DICT, config=psm_version))
    
    image_df = clean_df(image_df)
    
    return image_df


def process_coordinates(text_row):
    x = text_row['left'] + text_row['width']
    y = text_row['top'] + text_row['height']
    
    return x, y


#Now take image data and find word
def locate_text(text, df):
    stripped_text = "".join(text.lower().split())
    split_text = text.split()
    print(stripped_text, split_text)
    
    for i, row in df.iterrows():
        word = row['text']
        second_word = df.iloc[i+1]['text']
        
        
        if fuzz.ratio(word, stripped_text) == 80:
            print(fuzz.ratio(word, stripped_text))
            return "Found!", process_coordinates(row)
        
        if word == "endofdataframe":
            return "Not Found", (0, 0)
        
        if word in split_text[0] or split_text[0] in word: #may need to do partial_ratio(split_text[0], word)
            #Where we start fuzzy matching
            print(word)
            r1 = fuzz.ratio(word, stripped_text)
            r2 = fuzz.ratio((word + second_word), stripped_text)
            
            j = i
            while r2 >= r1:
                print("ITERATION")
                print(r2,r1)
                print(word, second_word)
                if r2 > 80:
                    print(second_word)
                    return "Found!", process_coordinates(row)
                
                word = word + df.iloc[j+1]['text']
                second_word = word + df.iloc[j+2]['text']
                
                print(word, second_word)
                
                
                r1 = r2
                r2 = fuzz.ratio(second_word, stripped_text)
                print(r2,r1)
                j+=1
                print("\n")
            print("end")
            
def text_coordinates(image_path, text):
    create_tesseract_path()
    
    df = process_image(image_path)
    status, coors = locate_text(text,df)
    
    return (status, coors)

In [4]:
image_path = r"C:\Users\sbuca\PYTORCH VIDEO\pierre\music_files\autogui_screenshots\spotify_screenshot.png"
text = "current screen png"
text_coordinates(image_path, text)

TesseractNotFoundError: \\Tesseract-OCR\tesseract.exe is not installed or it's not in your PATH. See README file for more information.